In [1]:
import numpy as np
import csv
import time

# Artist Information Processing

In [23]:
# load artists feature

artist_feature_file = 'artist features.csv'

artist_data = {}
maxArray=[]
with open(artist_feature_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        artist   = row[1]
        allInfo = [float(i) for i in row[2:]]

        artist_data[artist] = allInfo
        maxArray.append(allInfo)
maxArray=np.array(maxArray)
maxArray=np.amax(maxArray,axis=0)

for key in artist_data.keys():
    artist_data[key]=artist_data[key]/maxArray        

In [15]:
# Predict via the median number of plays.

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'global_median.csv'

# Load the training data.
train_data = {}

with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = int(row[2])
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = plays

In [2]:
# Predict via the median number of plays.

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'global_median.csv'

train_data=[]
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = int(row[2])
        
        train_data.append([user,artist, plays])

                          
train_data=np.array(train_data)
np.random.shuffle(train_data)
                          
train_test_ratio=90./100.
train_number=int(train_test_ratio*len(train_data))
test_number=len(train_data)-train_number

                
# Load the training data.
train_data_train = {}
train_data_test= {}

f = open('train_train.csv', 'w')
f.write('user,artist,plays\n')

for row in range(0,train_number):
    user   = train_data[row,0]
    artist = train_data[row,1]
    plays  = train_data[row,2]
    f.write('{},{},{}\n'.format(user,artist,plays))

    if not user in train_data_train:
        train_data_train[user] = {}

    train_data_train[user][artist] = plays
f.close()
    
train_data_test_file='train_test_input.csv'     

f1 = open('train_test_input.csv', 'w')
f1.write('Id,user,artist\n')

f2 = open('train_test_answers.csv', 'w')
f2.write('Id,plays\n')
idd=0
for row in range(train_number,train_number+test_number):
    user   = train_data[row,0]
    artist = train_data[row,1]
    plays  = train_data[row,2]
    idd+=1
    f1.write('{},{},{}\n'.format(idd,user,artist))
    f2.write('{},{}\n'.format(idd,plays))


    if not user in train_data_test:
        train_data_test[user] = {}

    train_data_test[user][artist] = plays
    
f1.close();f2.close()

In [30]:
print len(train_data_train.keys())
print len(train_data_test.keys())

233286
193527


In [16]:
artistSum={}
for user in train_data:
    for artist in train_data[user]:
        if not artist in artistSum:
            artistSum[artist]=float(train_data[user][artist])
        else: artistSum[artist]+=float(train_data[user][artist])

maxArtist=[]
for artist in artistSum:
    maxArtist.append(artistSum[artist])
maxArtist=max(maxArtist)
for artist in artistSum:
    artistSum[artist]=artistSum[artist]/maxArtist

In [17]:
assert len(artist_data[artistSum.keys()[2]])<55,'already ran this before'
for artist in artist_data:
    if artist in artistSum:
        data=artist_data[artist]
        summ=artistSum[artist]
        data=np.append(data,summ)
        artist_data[artist]=data
    else:
        data=artist_data[artist]
        data=np.append(data,0)
        artist_data[artist]=data

In [18]:
train_test_ratio=90./100.
train_number=int(train_test_ratio*len(artist_data))
test_number=len(artist_data)-train_number

artist_keys=np.array(artist_data.keys())

np.random.shuffle(artist_keys)

artist_data_train={}
for i in range(0,train_number):
    artist_data_train[artist_keys[i]]=artist_data[artist_keys[i]]

artist_data_test={}
for i in range(train_number,train_number+test_number):
    artist_data_test[artist_keys[i]]=artist_data[artist_keys[i]]

# User Infromation Processing

In [221]:
# load user features

user_feature_file = 'user features.csv'

user_data = {}
maxArray=[]
allage=[]
allNoAge=[]
with open(user_feature_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user = row[1]
        age = row[2]
        allNoAge.append(age)
        if age!='': 
            if float(age)>5. and float(age)<90.: allage.append(float(age))
        else: age='N'
        allInfo = [float(i) for i in row[3:]]
        allInfo.append(age)
        user_data[user] = allInfo


maxAge=max(allage)
allage=np.array(allage)
allage=allage/maxAge
meanAge=np.mean(allage)

#correcting age by romoving unknowns and assigning average
#also normalizing all values of the age
for ud in user_data.keys():
    udd=user_data[ud]
    uddd=udd[-1]
    if uddd=='N' or float(uddd)>=90. or float(uddd)<=5.: udd[-1]=meanAge 
    else: udd[-1]=float(udd[-1])/maxAge
    user_data[ud]=udd

In [270]:
print meanAge*90

24.7835998566


In [267]:
print np.mean(np.array(artist_data.values())[:,4])*90

8.63283185841


In [40]:
#Generate artist_matrix for features of artists
feature_num = len(artist_data[artistSum.keys()[2]])

artist_matrix = np.zeros([2000,feature_num])
k = 0
for artist in artist_data:
    for feat in xrange(feature_num):
        artist_matrix[k,feat] = artist_data[artist][feat]
    k += 1

user_num = len(train_data)
train_matrix = np.zeros([user_num,2000])

In [41]:
# print artist_data

In [248]:
#Count number of elements in each class
def class_counter(a):
    classnum = max(a)
    classcounter = np.zeros([classnum,1])
    for i in xrange(len(a)):
        for j in xrange(classnum):
            if a[i] == j:
                classcounter[j] += 1
    return classcounter
#Run clustering on artists
class_number_artist = 20
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=class_number_artist, random_state=0).fit(artist_matrix)

In [249]:
class_label = kmeans.labels_
print kmeans.labels_[1:12]
class_counts = class_counter(kmeans.labels_)
print class_counts

[ 0  0  9  0  8  1  0  2  0 11  0]
[[ 962.]
 [ 295.]
 [ 181.]
 [  26.]
 [  15.]
 [   1.]
 [   8.]
 [   7.]
 [  92.]
 [ 123.]
 [   1.]
 [ 281.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]]


In [251]:
print artist_matrix[1,:]
print kmeans.cluster_centers_
# print feature_num
# print feat
# print k

[ 0.          1.          0.          0.          0.05014749  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.0246806 ]
[[ -6.05071548e-15   1.00000000e+00  -3.27515792e-15   1.06858966e-15
    5.72492165e-02  -3.25260652e-19  -3.25260652e-19  -3.25260652e-19
   -3.25260652e-19   1.03950104e-03  -3.25260652e-19  -3.25260652e-19
   -3.25260652e-19  -3.25260652e-19  -3.25260652e-19  -3.25260652e-19
   -3.25260652e-19  -3.25260652e-19   1.03950104e-03  -3.25260652e-19
    1.03950104e-03  -3.25260652e-19  -3.25260652e-19  -3.25260652e-19
    1.03950104e-03  -3.25260652e

In [138]:
#Build an index dictionary for classes artists belonging to
artist_class = {}
k = 0
for artist in artist_data:
    if not artist in artist_class:
        artist_class[artist] = class_label[k]
    k=k+1

#Create user_matrix
origin_feature_num = len(user_data[user_data.keys()[1]])
feature_num = origin_feature_num + class_number_artist
user_matrix = np.zeros([user_num,feature_num])
k = 0
for user in user_data:
    for feat in xrange(origin_feature_num):
        user_matrix[k,feat] = user_data[user][feat]
    k += 1   

print user_matrix.shape

(233286, 262)


In [135]:
#compute the average time user takes for each class of artists
# User_class_times = np.zeros([user_num,class_number])
start_time = time.time()
count = 0
# for user in train_data:
#     count += 1
#     user_index = user_data.keys().index(user)
#     for j in xrange(class_number_artist):
#         summ = 0
#         for artist in train_data[user]:        
#             if artist_class[artist] == j:                
#                 summ += train_data[user][artist]
#         user_matrix[user_index][j+origin_feature_num] = summ
for user in train_data:
    count += 1
    user_index = user_data.keys().index(user)
    summ = 0
    #For every artist listened by this user
    for artist in train_data[user]:

        #Note:
        #Add to its artist class for this user
        #artist_class[artist] gives the class of this artist
        #0:origin_feature_num-1 means the basic features
        #origin_feature_num:end means the tendency features
        
        user_matrix[user_index][artist_class[artist]+origin_feature_num] += train_data[user][artist]
end_time = time.time()
print end_time - start_time

In [159]:
#Normalise the tendency
tendency_weight = 1
# for each user, tendency devided by user maximal
feature_max = 0

for i in xrange(user_num):
    row_max = max(user_matrix[i,origin_feature_num:])
    user_matrix[i,origin_feature_num:]/=row_max*tendency_weight
    

In [227]:
#clustering users
class_number = 500
from sklearn.cluster import KMeans
start_time = time.time()
kmeans_user = KMeans(n_clusters=class_number, random_state=0).fit(user_matrix[:,:])
end_time = time.time()
print end_time - start_time
class_label_user = kmeans_user.labels_
print kmeans_user.labels_[1:12]
class_counts = class_counter(class_label_user)
print class_counts

1855.83674717
[436 446 105  10 476 194  20 323 412 380 260]
[[  381.]
 [ 1811.]
 [  828.]
 [  399.]
 [ 1191.]
 [  876.]
 [  533.]
 [  865.]
 [  415.]
 [ 1587.]
 [  452.]
 [  680.]
 [ 1560.]
 [ 1558.]
 [  953.]
 [  727.]
 [  341.]
 [  604.]
 [  520.]
 [  516.]
 [ 1147.]
 [  204.]
 [  335.]
 [  519.]
 [  773.]
 [  798.]
 [  526.]
 [  483.]
 [  951.]
 [  213.]
 [  493.]
 [ 1165.]
 [ 1482.]
 [  746.]
 [  240.]
 [  439.]
 [ 1236.]
 [  244.]
 [  331.]
 [  271.]
 [  272.]
 [  719.]
 [  262.]
 [  498.]
 [  678.]
 [  561.]
 [  454.]
 [  436.]
 [  344.]
 [  351.]
 [  302.]
 [  200.]
 [  344.]
 [  335.]
 [  759.]
 [  617.]
 [  383.]
 [  601.]
 [  211.]
 [  970.]
 [  369.]
 [  289.]
 [  432.]
 [  421.]
 [  168.]
 [ 1623.]
 [ 1884.]
 [  531.]
 [  814.]
 [  753.]
 [  571.]
 [ 1330.]
 [  464.]
 [  454.]
 [  402.]
 [  585.]
 [  380.]
 [  265.]
 [  262.]
 [  361.]
 [  421.]
 [  158.]
 [  540.]
 [  252.]
 [  695.]
 [  480.]
 [  211.]
 [  556.]
 [  430.]
 [  301.]
 [  264.]
 [  818.]
 [ 1273.]
 [  283.]


In [237]:
# print kmeans.cluster_centers_

#Build an index dictionary for classes artists belonging to
user_class = {}
k = 0
for user in user_data:
    if not user in user_class:
        user_class[user] = class_label_user[k]
    k=k+1

In [231]:
#Create the core matrix
core = np.zeros([class_number,class_number_artist])
user_artist_class_count = np.zeros([class_number,1])
user_class_counts = np.zeros([class_number,1])
for k in xrange(user_num):
    user_class_counts[class_label_user[k]] += 1
    likeness_array = []
    j_array = []
    for j in xrange(class_number_artist):
        #class_label_user[k] gives the class number k-th user belongs to
        #for every user in this class, add artist playing times all together
        likeness = user_matrix[k,origin_feature_num+j]
        if likeness>0:
            likeness_array.append(likeness)
            j_array.append(j)
    likeness_array /= np.mean(likeness_array)
    for i in xrange(len(j_array)):
        core[class_label_user[k],j_array[i]] += likeness_array[i]

# #inside each user class, devide value for different artist class by mean
# #This produces tendency: if >1, like to listen more; if <1 listen less
core_tendency = np.zeros([class_number,class_number_artist])
scale = 0.1
for i in xrange(class_number):
    core_tendency[i,:] = (core[i,:]/user_class_counts[i]-1)*scale + 1

print core_tendency

# count1 = 0
# for i in xrange(class_number):
#     for j in xrange(class_number_artist):
#         if core_tendency[i,j]>0.5:
#             count1 += 1
# print count1

[[ 0.94324418  0.94561585  1.32871977 ...,  0.8         0.8         0.80007836]
 [ 1.31023188  1.03289286  0.83418737 ...,  0.8         0.8         0.80015499]
 [ 1.17294729  0.87920373  0.84131989 ...,  0.80031809  0.80023321
   0.80081761]
 ..., 
 [ 1.31160827  0.90399425  0.88907664 ...,  0.8         0.8         0.8       ]
 [ 1.1569743   0.92693023  0.91555827 ...,  0.8         0.80029363  0.8       ]
 [ 0.98877088  1.05438769  0.85467447 ...,  0.8         0.8         0.8       ]]


In [241]:
print np.max(core_tendency)

1.2172403368


In [236]:
#I take advantage of the User median file, add tendency to every user given artist group
#User median file
train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

# Load the training data.
train_data = {}
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = row[2]
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = int(plays)

# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)
    
    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]
            #I changed here
            if user in user_medians:
                i = user_class[user]
                j = artist_class[artist]
                #Add tendency to user_mean
                soln_csv.writerow([id, user_medians[user]])#*core_tendency[i,j]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])

# Calculating error of Test Data

In [4]:
def CalculateError(PredictionFileName):
    Prediction=np.loadtxt(PredictionFileName, delimiter=',')
    Data=np.loadtxt('train_test_answers.csv', delimiter=',')
    Err=0
    for i in xrange(len(Data[:,1])):
        Err += abs(Prediction[i,1]-Data[i,1])
    Err=Err/len(Data[:,1])
    print 'Error:',Err